In [1]:
import json
import pandas as pd
import numpy as np
from tqdm import tqdm

from gpt import AsyncYandexGpt, gen_fake_add

In [12]:
config = {}
with open('config.json') as f:
    config = json.load(f)

FOLDER_ID = config['FOLDER_ID']
API_KEY = config['API_KEY']
URI = f'gpt://{FOLDER_ID}/yandexgpt/latest'

In [13]:
gpt = AsyncYandexGpt(API_KEY, URI)

In [15]:
sample_text = """Москвичи чаще стали выбирать беспородных кошек, а среди породистых по-прежнему популярны шотландская вислоухая, британская короткошерстная и мейн-кун, говорится в сообщении на официальном сайте мэра и правительства Москвы.
Отмечается, что у владельцев собак популярны миниатюрные породы – например, йоркширские терьеры, чихуахуа и джек-рассел-терьеры.
Кроме того, у москвичей живут кролики, шиншиллы, рептилии и другие экзотические животные – их более 11 тысяч."""

In [ ]:
for i in range(3):
    res = gen_fake_add(gpt, sample_text)
    print(f'Sample {i+1}')
    print(res)
    print()

## Генерация датасета

In [3]:
lenta_ru = pd.read_csv("../data/lenta-ru-news.csv")

/var/folders/c1/6p_qy_fs0jj5z6y2p5nfx08w0000gn/T/ipykernel_36889/989240731.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  lenta_ru = pd.read_csv("../data/lenta-ru-news.csv")


In [4]:
list(zip(*np.unique(lenta_ru["topic"][lenta_ru["topic"].notna()].values, return_counts=True)))

[('69-я параллель', 1268),
 ('Библиотека', 65),
 ('Бизнес', 7399),
 ('Бывший СССР', 53402),
 ('Дом', 21734),
 ('Из жизни', 27605),
 ('Интернет и СМИ', 44663),
 ('Крым', 666),
 ('Культпросвет ', 340),
 ('Культура', 53797),
 ('Легпром', 114),
 ('МедНовости', 1),
 ('Мир', 136621),
 ('Наука и техника', 53136),
 ('Оружие', 3),
 ('Путешествия', 6408),
 ('Россия', 160445),
 ('Силовые структуры', 19596),
 ('Сочи', 1),
 ('Спорт', 64413),
 ('Ценности', 7766),
 ('ЧМ-2014', 2),
 ('Экономика', 79528)]

In [5]:
np.random.seed(42)

topics_for_gen = ["Дом", "Из жизни", "Интернет и СМИ", "Наука и техника", "Путешествия", "Спорт"]
gen_numbers = [333, 333, 333, 333, 334, 334]
indexes_for_gen = []
for topic, size in zip(topics_for_gen, gen_numbers):
    indexes = lenta_ru[lenta_ru["topic"] == topic]["text"].index
    indexes_for_gen += list(np.random.choice(indexes, size=size, replace=False))


In [10]:
data_text = lenta_ru.iloc[indexes_for_gen][["text"]].copy()
data_text["generated_text"] = None

In [10]:
def generate(row):
    return gen_fake_add(gpt, row["text"])

In [26]:
batch_size = 10

In [119]:
for i in tqdm(range(194, 200)):
    left = i * batch_size
    right = (i + 1) * batch_size
    data_text.iloc[left:right, 1] = data_text.iloc[left:right].apply(generate, axis=1)

100%|██████████| 6/6 [05:10<00:00, 51.79s/it]


In [124]:
check_na = data_text.iloc[0 * batch_size:200 * batch_size]["generated_text"]
print(check_na.isna().sum(), check_na.notna().sum())

627 1373


## Сохранение датасета

In [18]:
from dataclasses import make_dataclass

Text = make_dataclass("Text", [("title", str), ("text", str), ("label", int), ("url", str)])

In [126]:
texts = []
for index, row in data_text.iterrows():
    texts.append(Text(np.nan, row["generated_text"], 1, "Nadia"))

In [127]:
data_to_save = pd.DataFrame(texts, index=data_text.index)

In [128]:
data_to_save.to_csv("generated_data_by_news_p1")

In [129]:
data_text["generated_text"].isna().sum()

627

## Добавим немного к датасету

In [56]:
def generate_indexes(topics_for_gen, already_in_use: list[int], gen_numbers=[300, 300, 300, 300, 300, 300], seed=500):
    np.random.seed(seed)

    new_indexes_for_gen = []
    for topic, size in zip(topics_for_gen, gen_numbers):
        indexes = lenta_ru[lenta_ru["topic"] == topic]["text"].index
        indexes = [idx for idx in indexes if idx not in already_in_use]
        new_indexes_for_gen += list(np.random.choice(indexes, size=size, replace=False))

    return new_indexes_for_gen


In [57]:
new_indexes_for_gen = generate_indexes(topics_for_gen, indexes_for_gen)
batch_size = 10
new_data_text = lenta_ru.iloc[new_indexes_for_gen][["text"]].copy()
new_data_text["generated_text"] = None

In [21]:
new_data_text

,text,generated_text
637112,Американский актер Джонни Депп снизил стоимост...,None
406102,На 2012 год девелоперами анонсирован ввод в Мо...,None
732874,80 процентов жилых комплексов в России строятс...,None
603635,Новых квартир стоимостью пять и менее миллионо...,None
251167,"Казахстанская компания ""Астана-проект"" разрабо...",None
...,...,...
193729,Вторая ракетка мира россиянка Мария Шарапова в...,None
538872,Министр спорта России Виталий Мутко присвоил с...,None
641529,Российская теннисистка Анастасия Павлюченкова ...,None
308409,"На итоговом теннисном турнире года, стартовавш...",None


In [ ]:
for i in tqdm(range(91, 120)):
    left = i * batch_size
    right = (i + 1) * batch_size
    new_data_text.iloc[left:right, 1] = new_data_text.iloc[left:right].apply(generate, axis=1)

In [121]:
check_na = new_data_text.iloc[77 * batch_size:97 * batch_size]["generated_text"]
print(check_na.isna().sum(), check_na.notna().sum())

71 129


In [122]:
texts = []
for index, row in new_data_text.iterrows():
    texts.append(Text(np.nan, row["generated_text"], 1, "Nadia"))
data_to_save = pd.DataFrame(texts, index=new_data_text.index)

In [123]:
merge = pd.read_csv("generated_data_by_news_p2_1", index_col=0)

In [126]:
data_to_save.iloc[0 * batch_size:77 * batch_size] = merge.iloc[0 * batch_size:77 * batch_size]

In [128]:
check_na = data_to_save.iloc[0 * batch_size:97 * batch_size]
print(check_na.isna().sum(), check_na.notna().sum())

title    970
text     324
label      0
url        0
dtype: int64 title      0
text     646
label    970
url      970
dtype: int64


In [129]:
data_to_save.to_csv("generated_data_by_news_p2_2")

## Merge 2 files

In [130]:
data1 = pd.read_csv("generated_data_by_news_p1", index_col=0)
data2 = pd.read_csv("generated_data_by_news_p2", index_col=0)

In [135]:
all_data = pd.concat([data1, data2])

In [138]:
all_data["text"].notna().sum()

2019

In [140]:
data_to_save.to_csv("generated_data_by_news.csv", index=False)